# Robust Optimization in Portfolio Management

Author: Xiaochen (Lily) Wang, Chao (Kenneth) Wang

## Preliminary setting

In [1]:
using DataFrames, CSV # load data
using JuMP, Gurobi # modeling
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots # plot
using Distributions

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);

## Data

In [5]:
r = CSV.read("returns_stable.csv", DataFrame, header=true);
r = r[:, 2:11];

In [6]:
# mean return of each stock
mu = [mean(c) for c in eachcol(r)]

10-element Vector{Float64}:
 0.0026115411756341815
 0.0021749143451663265
 0.002194025834050713
 0.0017130032634385564
 0.0007245261442354494
 0.0005826240601159012
 0.0014216658358032193
 0.0018013303526384415
 0.0028287516071910023
 0.0008757915482220001

In [7]:
# covariance matrix
function covmat(df)
    nc = ncol(df)
    t = zeros(nc, nc)
    for (i, c1) in enumerate(eachcol(df))
        for (j, c2) in enumerate(eachcol(df))
            sx, sy = skipmissings(c1, c2)
            t[i, j] = cov(collect(sx), collect(sy))
        end
    end
    return t
    end;

sigma = covmat(r);

In [8]:
sigma

10×10 Matrix{Float64}:
 0.000273076  8.29e-5      0.000137522  …  5.77624e-5   5.5234e-5
 8.29e-5      0.000237286  2.3551e-5       1.56764e-5   3.59023e-5
 0.000137522  2.3551e-5    0.000456908     0.00012323   1.80563e-5
 0.000135602  7.97819e-5   0.000147016     6.15471e-5   6.09688e-5
 0.00013312   0.000108615  8.62121e-5      4.86934e-5   5.92861e-5
 4.38083e-5   1.69038e-5   2.04754e-5   …  3.61631e-5   3.28267e-5
 9.20725e-5   5.33401e-5   0.000100118     4.88441e-5   4.4062e-5
 0.000137431  8.4762e-5    7.04834e-5      3.07469e-5   5.37733e-5
 5.77624e-5   1.56764e-5   0.00012323      0.000441056  3.13844e-5
 5.5234e-5    3.59023e-5   1.80563e-5      3.13844e-5   0.000251488

In [9]:
sigma_inv = inv(sigma)

10×10 Matrix{Float64}:
  7250.77        37.2829   -882.584   …  -1828.02       8.29027    -5.50679
    37.2829    5944.01      543.666       -957.248     75.8049    100.894
  -882.584      543.666    3067.74          32.0389  -577.615     334.306
 -1940.41     -1368.92    -1485.34       -1154.23      59.4005   -742.787
 -2798.32     -2681.54     -164.402        626.105   -200.133    -814.784
 -1197.88       -43.071     228.971   …   -640.556   -634.812    -911.444
   841.098      959.901     -23.6363       648.168   -298.234      27.0806
 -1828.02      -957.248      32.0389      4603.93      41.2188   -351.442
     8.29027     75.8049   -577.615         41.2188  2529.99     -127.822
    -5.50679    100.894     334.306       -351.442   -127.822    4516.61

## Model

In [10]:
required_return = 0.0004
w_threshold = 0.3;

### Baseline

In [11]:
function train_baseline()
    model_base = Model(() -> Gurobi.Optimizer(GRB_ENV))

    I = 10;
    
    # decision variable
    @variable(model_base, w[1:I]>=0);
    
    # objective
    @objective(model_base, Min, sum(sigma[i, j] * w[i] * w[j] for i=1:I, j=1:I));
    
    # constraint
    @constraint(model_base, sum(mu[i] * w[i] for i=1:I) >= required_return);
    @constraint(model_base, threshold_constraint[i in 1:I], w[i] <= w_threshold);
    @constraint(model_base, sum(w[i] for i = 1:I) == 1);
 
    optimize!(model_base)
    
    w_opt = value.(w)
    
    return sqrt(objective_value(model_base)), w_opt
end

train_baseline (generic function with 1 method)

In [12]:
risk_bl, weight_bl = train_baseline();

In [13]:
risk_bl

0.007880130000626975

In [14]:
weight_bl

10-element Vector{Float64}:
 1.3118163452377615e-6
 0.15329428702782977
 0.0383907141176653
 3.740682673811959e-8
 0.016337438642738705
 0.2999999506301124
 0.23675419517297866
 0.04784659642949795
 0.06962707825487854
 0.13774839050112644

In [15]:
sum(mu .* weight_bl)

0.0013446292059672734

### Robust

In [16]:
function train_model(rho)
    model_robust = Model(() -> Gurobi.Optimizer(GRB_ENV))

    I = 10;
    
    # decision variable
    @variable(model_robust, w[1:I]);
    
    # objective
    @objective(model_robust, Min, sum(sigma[i, j] * w[i] * w[j] for i=1:I, j=1:I));
    
    # constraint
    @constraint(model_robust, rho^2 * sum(sigma_inv[i, j] * w[i] * w[j] for i=1:I, j=1:I) 
        <= (sum(mu[i] * w[i] for i=1:I) - required_return)^2);
    @constraint(model_robust, sum(mu[i] * w[i] for i=1:I) - required_return >= 0);
    @constraint(model_robust, threshold_constraint[i in 1:I], w[i] <= w_threshold);
    @constraint(model_robust, sum(w[i] for i = 1:I) == 1);

    optimize!(model_robust)
    
    w_opt = value.(w)
    
    return sqrt(objective_value(model_robust)), w_opt
end

train_model (generic function with 1 method)

In [22]:
risk_ro, weight_ro = train_model(0.00019);

In [23]:
risk_ro

0.04230873303905182

In [24]:
weight_ro

10-element Vector{Float64}:
 0.13543925852197983
 0.0718159907155095
 0.16127392881843355
 0.12826600741817146
 0.10824482346132544
 0.034816756013454936
 0.0916356753972918
 0.10293417422847334
 0.11234766318641047
 0.05322572223895195

In [25]:
sum(mu .* weight_ro)

0.0018622814005557762

#### Get feasible $\rho$

In [26]:
# for required_return = 0:0.0001:0.0005
# for w_threshold = 0.3:0.1:1

rho_list = zeros(0)

for rho = 0:0.00001:0.006
    try
        risk, returns = train_model(rho)
        append!(rho_list, rho)
    catch
    end
end

length(rho_list)

57

In [27]:
n = length(rho_list)

risk_list = zeros(n,)
weight_list = zeros(n, 10)
return_list = zeros(n,)
for i = 1:n
    risk, weight = train_model(rho_list[i])
    risk_list[i] = risk
    weight_list[i, :] = weight
end

### Feasibility trade off

In [28]:
test = CSV.read("returns_covid.csv", DataFrame, header=true);
test = test[:, 2:11];

In [29]:
test_size = size(test)[1]

61

#### Baseline vs Single Robust (Sanity Check)

In [31]:
feasible_count_bl = 0

for i = 1:test_size
    new_r = Vector(test[i, :])
   
    if sum(new_r .* weight_bl) >= required_return
       feasible_count_bl = feasible_count_bl + 1
    end

end

println("baseline: ", feasible_count_bl/test_size)

baseline: 0.3770491803278688


##### Baseline vs Robust with Different Rho

In [32]:
bl_feasible_count_list = zeros(n)
bl_feasible_count_list .= feasible_count_bl/test_size;

In [34]:
ro_feasible_count_list = zeros(n)

for rho_idx = 1:n
    cur_weight = weight_list[rho_idx, :]
    
    for i = 1:size(test)[1]
        new_r = Vector(test[i, :])

        if sum(new_r .* cur_weight) >= required_return
           ro_feasible_count_list[rho_idx] = ro_feasible_count_list[rho_idx] + 1
        end
        
    end
end

In [35]:
risk_bl_list = zeros(n)
risk_bl_list .= round(risk_bl, digits = 6);

In [36]:
df = zeros(n, 5);

df[:, 1] = rho_list; 
df[:, 2] = risk_bl_list
df[:, 3] = risk_list
df[:, 4] = bl_feasible_count_list;
df[:, 5] = ro_feasible_count_list/test_size;

In [37]:
df[:, :]

57×5 Matrix{Float64}:
 0.00018  0.00788  0.0102758   0.377049  0.442623
 0.00019  0.00788  0.0423087   0.377049  0.442623
 0.00026  0.00788  0.00947982  0.377049  0.409836
 0.0003   0.00788  0.0645611   0.377049  0.442623
 0.00032  0.00788  0.058315    0.377049  0.442623
 0.00043  0.00788  0.155997    0.377049  0.442623
 0.00044  0.00788  0.174328    0.377049  0.42623
 0.00045  0.00788  0.196952    0.377049  0.42623
 0.00046  0.00788  1.20172e13  0.377049  0.442623
 0.00053  0.00788  0.092211    0.377049  0.442623
 0.00059  0.00788  0.114688    0.377049  0.442623
 0.0006   0.00788  0.108113    0.377049  0.442623
 0.00063  0.00788  0.645433    0.377049  0.42623
 ⋮                                       
 0.00523  0.00788  0.334505    0.377049  0.442623
 0.00524  0.00788  0.334104    0.377049  0.442623
 0.00525  0.00788  0.336105    0.377049  0.442623
 0.00526  0.00788  0.336708    0.377049  0.442623
 0.00529  0.00788  0.339095    0.377049  0.442623
 0.0053   0.00788  0.339725    0.377049

In [38]:
using DelimitedFiles
writedlm("robust_rho.csv",  df, ',')

In [39]:
weight_bl = [round(w, digits = 3) for w in weight_bl]

10-element Vector{Float64}:
 0.0
 0.153
 0.038
 0.0
 0.016
 0.3
 0.237
 0.048
 0.07
 0.138

In [40]:
weight_ro = [round(w, digits = 3) for w in weight_list[1, :]]

10-element Vector{Float64}:
 0.133
 0.083
 0.145
 0.125
 0.101
 0.037
 0.089
 0.107
 0.119
 0.061

In [41]:
sum(weight_bl .* mu)

0.0013447850190169272

In [42]:
sum(weight_ro .* mu)

0.001864161584483464

In [43]:
 # mean return of each stock
test_mu = [mean(c) for c in eachcol(test)]
test_sigma = covmat(test)
test_std = [sqrt(test_sigma[i, i]) for i = 1:10];

In [44]:
test_mu

10-element Vector{Float64}:
 -0.0018540586434201913
 -0.009614645376845612
 -0.005921801167942255
 -0.008921102843116882
 -0.009601225615125705
 -0.0012489489420897863
 -0.006090971134191329
 -0.0030250566710084926
 -0.004358122729417964
 -0.0013705709473801103

In [45]:
test_std

10-element Vector{Float64}:
 0.042018878496089825
 0.07524643800139028
 0.045995450866070535
 0.05972447406334571
 0.06609958874458936
 0.03221018338895366
 0.05031459876089027
 0.038051379740848476
 0.035389532000139126
 0.0498191469310034

In [46]:
test_mu./test_std

10-element Vector{Float64}:
 -0.04412442001736733
 -0.12777542209596643
 -0.12874754038579475
 -0.14937097367579782
 -0.14525393875331216
 -0.038774971474335285
 -0.12105773044394948
 -0.07949926366956594
 -0.12314722696533063
 -0.027510927661572985

### Simulation

In [ ]:
mu = [mean(c) for c in eachcol(r)];
r_std = [sqrt(sigma[i, i]) for i = 1:10];

n_samples = 100
r_uncertain_set = zeros(n_samples, 10);

for i = 1:10
    d = Normal(mu[i], r_std[i])
    r_uncertain_set[:, i] = rand(d, n_samples)
end

In [ ]:
# simulation
ro_feasible_count_list = zeros(n)

for rho_i = 1:n
    cur_weight = weight_list[rho_i, :]
    
    for sample = 1:n_samples
        new_r = r_uncertain_set[sample, :]

        if sum(new_r .* cur_weight) >= required_return
           ro_feasible_count_list[rho_i] = ro_feasible_count_list[rho_i] + 1
        end
        
    end
end